Note, this is some general noodling around with the multiply data.

Nothing much of note here

In [ ]:
# initial setup
import sys
sys.path.append("../analysis")

In [2]:
import parsing
import extraction

from parsing import parse_file
from extraction import humanize_names, expand_target_checks, pull_spec_in

In [3]:
%matplotlib widget

In [4]:
import pandas as pd
import numpy as np

import itertools

import matplotlib.pyplot as plt

In [5]:
def fetch(dataset):
    return humanize_names(parse_file(f"../results/{dataset}"))

In [6]:
raw_data = fetch("2020-04-26-timings-for-initial-quals-eval")
stencil_5_raw_data = fetch("2020-04-27-2d-stencil-5-timings")

In [7]:
data_3 = extraction.matrix_stats(raw_data['l3/2d-stencil-3'])
data_5 = extraction.matrix_stats(stencil_5_raw_data['l3/2d-stencil-5'])

In [8]:
data_3

,category,key,density,time
0,load,"matrices/4,4-identity-4,4,3,3",0.003906,0.001867
1,build,"matrices/4,4,3,3-2s-r{main[1]out[1]second[3]}-...",0.000217,0.023763
2,mul,"matrices/4,4-identity_4,4,3,3-2s-r{main[1]out[...",0.017578,1.325192
3,build,"matrices/4,4,3,3-2s-xf{main[1]out[1]second[3]}...",0.000315,0.036831
4,mul,"matrices/4,4-identity_4,4,3,3-2s-r{main[1]out[...",0.050456,3.800430
5,build,"matrices/4,4,3,3-2s-r{main[0]out[0]second[2]}-...",0.000217,0.023861
6,mul,"matrices/4,4-identity_4,4,3,3-2s-r{main[1]out[...",0.227051,1.786516
7,build,"matrices/4,4,3,3-2s-xf{main[0]out[0]second[2]}...",0.000315,0.037701
8,mul,"matrices/4,4-identity_4,4,3,3-2s-r{main[1]out[...",0.651720,1.788620
9,load,"matrices/4,4,3,3-2s-g{axis[3]dims[1|3]}-4,4,3,2",0.000416,0.065753


In [9]:
data_5

,category,key,density,time
0,load,"matrices/4,4-identity-4,4,5,5",0.003906,0.018067
1,build,"matrices/4,4,5,5-2s-r{main[1]out[1]second[3]}-...",0.000028,0.377566
2,mul,"matrices/4,4-identity_4,4,5,5-2s-r{main[1]out[...",0.017578,81.277744
3,build,"matrices/4,4,5,5-2s-xf{main[1]out[1]second[3]}...",0.000042,0.635898
4,mul,"matrices/4,4-identity_4,4,5,5-2s-r{main[1]out[...",0.051875,462.162612
5,build,"matrices/4,4,5,5-2s-r{main[0]out[0]second[2]}-...",0.000028,0.606631
6,mul,"matrices/4,4-identity_4,4,5,5-2s-r{main[1]out[...",0.233437,384.749273
7,build,"matrices/4,4,5,5-2s-xf{main[0]out[0]second[2]}...",0.000042,0.756580
8,mul,"matrices/4,4-identity_4,4,5,5-2s-r{main[1]out[...",0.688900,332.450648
9,build,"matrices/4,4,5,5-2s-g{axis[3]dims[1|3]}-4,4,5,2",0.000152,1.177773


In [10]:
multiply_times_3 = data_3[data_3['category'] == 'mul']['time']
multiply_times_3

2     1.325192
4     3.800430
6     1.786516
8     1.788620
10    0.791570
Name: time, dtype: float64

In [11]:
multiply_times_5 = data_5[data_5['category'] == 'mul']['time']
multiply_times_5

2      81.277744
4     462.162612
6     384.749273
8     332.450648
10     60.786790
Name: time, dtype: float64

In [12]:
multiply_times_5 / multiply_times_3

2      61.332809
4     121.607989
6     215.362886
8     185.869866
10     76.792737
Name: time, dtype: float64

In [13]:
# The sizes
out_size = 16 ** 2
in_size_3 = (4 * 4 * 3 * 3) ** 2
in_size_5 = (4 * 4 * 5 * 5) ** 2
(in_size_5 / in_size_3)

7.716049382716049

In [14]:
mul_ratio = multiply_times_5.sum() / multiply_times_3.sum()
mul_ratio

139.2100126940718

In [15]:
np.log(mul_ratio) / np.log(in_size_5 / in_size_3)

2.4156891539607557

Ok so that's $O(n^{2.4})$ empirically from two data points.
I can put better together from Trove

In [16]:
mul_big_o = np.log(mul_ratio) / np.log((16 * 25) / (16 * 9)) # Array elements
mul_big_o

4.831378307921512

In [17]:
25 / 9

2.7777777777777777

In [18]:
2 ** mul_big_o

28.47015234520906

The result, which we can try and confirm more carefully: if we double an input dimension, the multiplication sohuld get somewhere around 30x slower